# 1. Install dependencies

In [1]:
!pip install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Download the pre-trained model from 'Hugging Face'

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Testing the sentiment

In [8]:
tokens = tokenizer.encode('Good food, but the ambience could have been better', return_tensors='pt')

In [9]:
result = model(tokens)

In [10]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5120, -0.2941,  2.2432,  1.8703, -1.1405]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [11]:
result.logits

tensor([[-2.5120, -0.2941,  2.2432,  1.8703, -1.1405]],
       grad_fn=<AddmmBackward>)

In [12]:
int(torch.argmax(result.logits))+1

3

# 4. Web scrapping the reviews

In [13]:
r = requests.get('https://www.yelp.de/biz/freddy-schilling-köln')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [14]:
results

[<p class="comment__373c0__1M-px css-n6i4z7"><span class="raw__373c0__3rcx7" lang="de">Haben vor etwa zwei Wochen bestellt und die Bestellung war pünktlich fertig, noch warm und alles schmeckte super!! Lob an das Personal und bestelle dort immer wieder gerne!</span></p>,
 <p class="comment__373c0__1M-px css-n6i4z7"><span class="raw__373c0__3rcx7" lang="de">Für alle die Freddy Schilling in Köln noch nicht ausprobiert haben, tut es. Ich bin immer wieder begeistert wie gut alles schmeckt. Meine persönliche Empfehlung ist ganz klar der Brunos Brenner mit extra Speck und Käse und dazu ne große Pommes mit Kahuna Sauce und der Habanero Ketchup um für den großen brennenden Mundspaß (die ist wirklich sehr scharf!). In der Kyff geht es deutlich schneller als am Eigelsteintor und bei Freddy hat die Corona Maßnahme den Bestellprozess auf jedenfall Nachhaltig gut optimiert. Ich bestelle immer Online vor und hole es dann ab. Ich stand oft genug 30-40 Minuten vor dem Laden am Eigelstein wo es eigentl

# 5. Load Reviews into DataFrame and Score

In [16]:
import numpy as np
import pandas as pd

In [17]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [18]:
df['review'].iloc[0]

'Haben vor etwa zwei Wochen bestellt und die Bestellung war pünktlich fertig, noch warm und alles schmeckte super!! Lob an das Personal und bestelle dort immer wieder gerne!'

In [19]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [20]:
sentiment_score(df['review'].iloc[1])

5

In [21]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [22]:
df

,review,sentiment
0,Haben vor etwa zwei Wochen bestellt und die Be...,5
1,Für alle die Freddy Schilling in Köln noch nic...,5
2,Das ist das Original. Beste Burger in Köln fin...,5
3,"Preise sind angemessen, und Burger sehr lecker...",5
4,Leckere Burger frisch zubereitet. Pommes waren...,4
5,Das ist nicht mein erster Besuch und sicher ni...,2
6,Ich find's geil! Endlich wieder in Köln aber w...,4
7,Habe von mehreren Personen positives sowie neg...,2
8,"Wow, da hat mich die YelpApp zu einem wirklich...",4
9,Ich habe mir den Favorit bestellt mit Pommes (...,4


In [24]:
df['review'].iloc[5]

'Das ist nicht mein erster Besuch und sicher nicht mein Letzter.Mein letzter war im November, genau, am 30.11.2015. Das war ein schneller Wildburger mit einer Freundin, kurz vor einem Tattoo Termin. Ich war damals schon beendruckt. Superfrisch, das Fleisch perfekt gegart und unglaublich schmackhaft.Nun waren wir wieder einmal in der Nähe (am 13.03.) , naja, Breite Str, aber da gibt es nichts kulinarisch hochwertiges, und hatten Hunger...den nächsten Car to go Smart geschnappt und ins Kwartier Latäng. Freddy Schilling, wir kommen.Voll war es, aber Platz ist komischerweise immer. Die Jungs und Mädels waren wieder einmal sehr freundlich, tattoowiert und bemützt. Passt also gut zum Image. Dieses Mal gab es den Big Kahuna Burger...es war uns nach Urlaub unter Palmen.Und wie immer wurden wir nicht enttäuscht. Exzellent zubereitet und perfekt gegart. Die Zutaten wunderbar aufeinander abgestimmt...ein perfekter fruchtiger Burger. Ich mag es etwas schärfer, habe ich aber vergessen zu bestellen.